### Get environment name from widget and assign to variable 

In [0]:
dbutils.widgets.text(name="env",defaultValue='',label='Enter the environment in lower case')
env = dbutils.widgets.get("env")


<br>

### Call common notebook to access shared variables and methods

In [0]:
%run "/Workspace/Users/yen#ext#@yenyahoo.onmicrosoft.com/3_Common"


### Read Silver_Traffic table

In [0]:

def read_SilverTrafficTable(environment):

    print('Reading the Silver Traffic Table Data : ', end='')

    df_SilverTraffic = (spark.readStream
                    .table(f"`{environment}_catalog`.`silver`.silver_traffic")
                    )
    
    print(f'Reading {environment}_catalog.silver.silver_traffic Success!')

    print("**********************************")

    return df_SilverTraffic


### Read silver_roads Table

In [0]:

def read_SilverRoadsTable(environment):

    print('Reading the Silver Table Silver_roads Data : ', end='')

    df_SilverRoads = (spark.readStream
                    .table(f"`{environment}_catalog`.`silver`.silver_roads")
                    )
    
    print(f'Reading {environment}_catalog.silver.silver_roads Success!')

    print("**********************************")

    return df_SilverRoads


## Creating vehicle Intensity Column

In [0]:
def create_VehicleIntensity(df):
 
 print('Creating Vehicle Intensity column : ', end='')

 from pyspark.sql.functions import col

 df_veh = df.withColumn('Vehicle_Intensity',
               col('Motor_Vehicles_Count') / col('Link_length_km')
               )
 
 print("Success!!!")
 print('***************')
 
 return df_veh

### Creating LoadTime column

In [0]:
def create_LoadTime(df):
    
    from pyspark.sql.functions import current_timestamp

    print('Creating Load Time column : ',end='')

    df_timestamp = df.withColumn('Load_Time', current_timestamp() )
    
    print('Success!!')
    print('**************')
    return df_timestamp


### Writing Data to Gold Traffic


In [0]:
def write_Traffic_GoldTable(StreamingDF,environment):
    print('Writing the gold_traffic Data : ',end='') 

    write_gold_traffic = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation', checkpoint + "GoldTrafficLoad/Checkpt/")
                .outputMode('append')
                .queryName("GoldTrafficWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}_catalog`.`gold`.`gold_traffic`"))
    
    write_gold_traffic.awaitTermination()
    print(f'Writing `{environment}_catalog`.`gold`.`gold_traffic` Success!')


### Writing Data to Gold Roads


In [0]:
def write_Roads_GoldTable(StreamingDF,environment):
    print('Writing the gold_roads Data : ',end='') 

    write_gold_roads = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation', checkpoint + "GoldRoadsLoad/Checkpt/")
                .outputMode('append')
                .queryName("GoldRoadsWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}_catalog`.`gold`.`gold_roads`"))
    
    write_gold_roads.awaitTermination()
    print(f'Writing `{environment}_catalog`.`gold`.`gold_roads` Success!')


## Calling all functions


In [0]:
## Reading from Silver tables
df_SilverTraffic = read_SilverTrafficTable(env)
df_SilverRoads = read_SilverRoadsTable(env)
    
## Tranformations     
df_vehicle = create_VehicleIntensity(df_SilverTraffic)
df_FinalTraffic = create_LoadTime(df_vehicle)
df_FinalRoads = create_LoadTime(df_SilverRoads)


## Writing to gold tables    
write_Traffic_GoldTable(df_FinalTraffic, env)
write_Roads_GoldTable(df_FinalRoads, env)

Reading the Silver Traffic Table Data : Reading dev_catalog.silver.silver_traffic Success!
**********************************
Reading the Silver Table Silver_roads Data : Reading dev_catalog.silver.silver_roads Success!
**********************************
Creating Vehicle Intensity column : Success!!!
***************
Creating Load Time column : Success!!
**************
Creating Load Time column : Success!!
**************
Writing the gold_traffic Data : Writing `dev_catalog`.`gold`.`gold_traffic` Success!
Writing the gold_roads Data : Writing `dev_catalog`.`gold`.`gold_roads` Success!


### After adding new data

In [0]:
%sql

SELECT count(*) as `Num of Rows` FROM `dev_catalog`.`gold`.`gold_traffic` Limit 5

Num of Rows
55638


In [0]:
%sql

SELECT count(*) as `Num of Rows` FROM `dev_catalog`.`gold`.`gold_roads`

Num of Rows
76


### Display small sample of data

In [0]:
%sql

SELECT * FROM `dev_catalog`.`gold`.`gold_traffic` Limit 5

Record_ID,Count_point_id,Direction_of_travel,Year,Count_date,hour,Region_id,Region_name,Local_authority_name,Road_name,Road_Category_ID,Start_junction_road_name,End_junction_road_name,Latitude,Longitude,Link_length_km,Pedal_cycles,Two_wheeled_motor_vehicles,Cars_and_taxis,Buses_and_coaches,LGV_Type,HGV_Type,EV_Car,EV_Bike,Extract_Time,Electric_Vehicles_Count,Motor_Vehicles_Count,Transformed_Time,Vehicle_Intensity,Load_Time
18629,7903,E,2014,6/9/2014 0:00,15,7,East of England,Hertfordshire,M25,1,20,21,51.71406567,-0.40920569,4.7,0,13,3491,12,1180,182,38,38,2024-09-27T11:26:31.45Z,76,4954,2024-09-28T03:06:42.99Z,1054.0425531914893,2024-09-28T06:00:42.598Z
18744,36064,S,2014,5/8/2014 0:00,11,11,North East,Durham,A1(M),1,63/A167,LA Boundary,54.87638616,-1.565164323,1.9,0,5,1744,8,384,85,27,27,2024-09-27T11:26:31.45Z,54,2280,2024-09-28T03:06:42.99Z,1200.0,2024-09-28T06:00:42.598Z
18746,36064,N,2014,5/8/2014 0:00,13,11,North East,Durham,A1(M),1,63/A167,LA Boundary,54.87638616,-1.565164323,1.9,0,4,1830,10,417,65,23,54,2024-09-27T11:26:31.45Z,77,2403,2024-09-28T03:06:42.99Z,1264.7368421052631,2024-09-28T06:00:42.598Z
18756,36064,N,2014,5/8/2014 0:00,11,11,North East,Durham,A1(M),1,63/A167,LA Boundary,54.87638616,-1.565164323,1.9,0,6,1846,16,440,72,28,37,2024-09-27T11:26:31.45Z,65,2445,2024-09-28T03:06:42.99Z,1286.842105263158,2024-09-28T06:00:42.598Z
18827,20511,N,2014,5/20/2014 0:00,16,4,Wales,Monmouthshire,A40,3,B4293,A466,51.80987719,-2.712139559,0.8,0,1,920,14,202,23,9,0,2024-09-27T11:26:31.45Z,9,1169,2024-09-28T03:06:42.99Z,1461.25,2024-09-28T06:00:42.598Z


In [0]:
%sql

SELECT * FROM `dev_catalog`.`gold`.`gold_traffic` Limit 5

Record_ID,Count_point_id,Direction_of_travel,Year,Count_date,hour,Region_id,Region_name,Local_authority_name,Road_name,Road_Category_ID,Start_junction_road_name,End_junction_road_name,Latitude,Longitude,Link_length_km,Pedal_cycles,Two_wheeled_motor_vehicles,Cars_and_taxis,Buses_and_coaches,LGV_Type,HGV_Type,EV_Car,EV_Bike,Extract_Time,Electric_Vehicles_Count,Motor_Vehicles_Count,Transformed_Time,Vehicle_Intensity,Load_Time
18629,7903,E,2014,6/9/2014 0:00,15,7,East of England,Hertfordshire,M25,1,20,21,51.71406567,-0.40920569,4.7,0,13,3491,12,1180,182,38,38,2024-09-27T11:26:31.45Z,76,4954,2024-09-28T03:06:42.99Z,1054.0425531914893,2024-09-28T06:00:42.598Z
18744,36064,S,2014,5/8/2014 0:00,11,11,North East,Durham,A1(M),1,63/A167,LA Boundary,54.87638616,-1.565164323,1.9,0,5,1744,8,384,85,27,27,2024-09-27T11:26:31.45Z,54,2280,2024-09-28T03:06:42.99Z,1200.0,2024-09-28T06:00:42.598Z
18746,36064,N,2014,5/8/2014 0:00,13,11,North East,Durham,A1(M),1,63/A167,LA Boundary,54.87638616,-1.565164323,1.9,0,4,1830,10,417,65,23,54,2024-09-27T11:26:31.45Z,77,2403,2024-09-28T03:06:42.99Z,1264.7368421052631,2024-09-28T06:00:42.598Z
18756,36064,N,2014,5/8/2014 0:00,11,11,North East,Durham,A1(M),1,63/A167,LA Boundary,54.87638616,-1.565164323,1.9,0,6,1846,16,440,72,28,37,2024-09-27T11:26:31.45Z,65,2445,2024-09-28T03:06:42.99Z,1286.842105263158,2024-09-28T06:00:42.598Z
18827,20511,N,2014,5/20/2014 0:00,16,4,Wales,Monmouthshire,A40,3,B4293,A466,51.80987719,-2.712139559,0.8,0,1,920,14,202,23,9,0,2024-09-27T11:26:31.45Z,9,1169,2024-09-28T03:06:42.99Z,1461.25,2024-09-28T06:00:42.598Z
